In [ ]:
# Install dependencies and packages
!pip install sentencepiece
!pip install transformers
!pip install contractions

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.3 MB 26.2 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.5 MB 28.1 MB/s 
     |████████████████████████████████| 7.6 MB 54.2 MB/s 
     |████████████████████████████████| 163 kB 59.5 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 106 kB 17.2 MB/s 
     |████████████████████████████████| 287 kB 47.7 MB/s 


In [ ]:
# Check that GPU is available
import torch

print("GPU Available: {}".format(torch.cuda.is_available()))
n_gpu = torch.cuda.device_count()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Number of GPU Available: {}".format(n_gpu))
#print("GPU: {}".format(torch.cuda.get_device_name(0)))

GPU Available: True
Number of GPU Available: 1


In [ ]:
# Import libraries
from google.colab import drive
import pandas as pd
import numpy as np
import re
import pickle
import nltk
from nltk.tokenize import TweetTokenizer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
import contractions
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('stopwords')
print("Libraries have been successfully imported")

Libraries have been successfully imported


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
# Permission to read and write for own Google Drive
drive.mount('/content/drive')

# Ensure you make a shortcut of folder on current Google Drive account
# Set the paths accordingly to the file path
training_set_path = "/content/drive/MyDrive/BT4103 CAPSTONE/Data/assets/sensitised_covid_narrative_training_dataset.xlsx"

emoji_dict_path = "/content/drive/MyDrive/BT4103 CAPSTONE/Data/assets/emoji.xlsx"
emoticon_dict_path = "/content/drive/MyDrive/BT4103 CAPSTONE/Data/assets/emoticons.xlsx"
nonenglish_dict_path = "/content/drive/MyDrive/BT4103 CAPSTONE/Data/assets/nonenglish.xlsx"

Mounted at /content/drive


In [ ]:
# Import Training dataset
train = pd.read_excel(training_set_path)
train

,text,label
0,"Of Course Dr Goh is in the wrong, Only their E...",AlternativeCures
1,There's also the medicine Ivermectin that is u...,AlternativeCures
2,"Try googling for ""Iris Koh Church""\n\nMs Iris ...",AlternativeCures
3,Neo Ko Hwai Ivan i know of covid cases who too...,AlternativeCures
4,Just take some cough syrup they will be up and...,AlternativeCures
...,...,...
1014,"王望星 HIV, a similar RNA virus in structure to C...",VirusMisinformation
1015,"""n aft more thn 80% vaccinated...nw\n5000 dail...",Vaccines_HealthRisk
1016,"well prayer does work, My friend had cancer an...",Others
1017,Tan Kia Sin This database include entries that...,Vaccines_HealthRisk


In [ ]:
# Function to read excel files and store in dictionaries to be used in clean function later
def df_to_dict(storage, cols):
  dct = {}
  for index, row in storage.iterrows():
    dct[row[cols[0]]] = row[cols[1]]
  return dct

emojis = pd.read_excel(emoji_dict_path)
emojis_dict = df_to_dict(emojis, ['Emoji', 'Meaning'])

emoticons = pd.read_excel(emoticon_dict_path)
emoticons_dict = df_to_dict(emoticons, ['Emoticon', 'Meaning'])

nonenglish = pd.read_excel(nonenglish_dict_path)
nonenglish_dict = df_to_dict(nonenglish, ["Word", "Meaning"])

In [ ]:
# creater tokenizer and lemmatizer for the clean function later
tokenizer = TweetTokenizer(preserve_case=False, strip_handles=True)
lemmatizer = WordNetLemmatizer()
stop = set(stopwords.words('english'))

In [ ]:
# Text pre-processing steps
def clean(text):
  tokenised = tokenizer.tokenize(contractions.fix(text)) # Tokenise text and replace contractions
  tokenised = list(map(lambda x: x if x not in emojis_dict else emojis_dict[x], tokenised)) # replace emoji with text
  tokenised = list(map(lambda x: x if x not in emoticons_dict else emoticons_dict[x], tokenised)) # replace emoticon with text
  tokenised = list(filter(lambda x: not (bool(re.match("\B\#\S+ |\B\#\S+", x) or bool(re.match("[a-zA-Z0-9-_.]+@[a-zA-Z0-9-_.]+", x)) or bool(re.match("(?:https?|ftp):\/\/[\w\/\-?=%.]+\.[\w\/\-&?=%.]+", x)) or bool(re.match("(?:https ?|ftp):\/\/[\w\/\-?=%.]+\.[\w\/\-&?=%.]+", x)) or bool(re.match("(?:https?|ftp):\/\/ [\w\/\-?=%.]+\.[\w\/\-&?=%.]+", x)))), tokenised)) # remove hashtags and hyperlinks
  tokenised = list(filter(lambda x: x.isalpha(), tokenizer.tokenize(" ".join(tokenised)))) # keep only alphabets
  tokenised = list(map(lambda x: x if x not in nonenglish_dict else nonenglish_dict[x], tokenised))
  lemmatized = [lemmatizer.lemmatize(token) for token in tokenizer.tokenize(" ".join(tokenised))] # lemmatized
  cleaned = list(filter(lambda x: x not in stop, lemmatized)) # remove stop words
  #print(cleaned)
  return " ".join(cleaned)

In [ ]:
# Running text column through the clean text function 
train["text"] = train["text"].apply(lambda x: clean(x) if x is not np.nan else x)
train

,text,label
0,course doctor goh wrong experimental gene ther...,AlternativeCures
1,also medicine antiparastic drug used india mex...,AlternativeCures
2,try googling iris koh church iris koh said ha ...,AlternativeCures
3,neo ko hwai ivan know covid case took med plus...,AlternativeCures
4,take cough syrup day,AlternativeCures
...,...,...
1014,王望星 hiv similar rna virus structure coronaviru...,VirusMisinformation
1015,n aft thn vaccinated nw daily covid cs n death...,Vaccines_HealthRisk
1016,well prayer doe work friend cancer going heali...,Others
1017,tan kia sin database include entry seem made s...,Vaccines_HealthRisk


In [ ]:
# Set the paths accordingly to the file path, this is for the BERT labelled excel
test_set_path = "/content/drive/MyDrive/BT4103 CAPSTONE/Data/assets/sensitised_covid_narrative_dataset_labelled.xlsx"
test = pd.read_excel(test_set_path)

In [ ]:
# Running text column through the clean text function 
test["text"] = test["text"].apply(lambda x: clean(x) if x is not np.nan else x)
test

,Unnamed: 0,group,post_id,text,comment_id,comment_time,type,comment_pred_prob,label
0,0.0,0.0,b'gAAAAABjLvrbdzkEZejvmMaD3jhEW2xLa2rdlF12CRFT...,hi healing divide affected negatively censored...,b'gAAAAABjLvrbMz1DP9NuKqNF1sKwklclYXB77_APPpbG...,2021-11-24 18:53:08,comment,0.999825,Others
1,1.0,0.0,b'gAAAAABjLvrb0Npi7ZGe_rQcZQNjEG_HD7t1r9TUO92T...,father god watching going earth per kat kerr n...,b'gAAAAABjLvrbdaF_OuS8k4JPxBCmGJqZ62nxlmwyO7Al...,2021-11-24 18:53:11,comment,0.999054,Conspiracy_Control&Manip
2,2.0,0.0,b'gAAAAABjLvrb3W3z--AsrO1si5aKydeAmrNbglLKjVCD...,toto teosinte,b'gAAAAABjLvrb9Siosaam0Fd45-MbpDWCnzffwoOxN7a8...,2021-11-24 18:53:11,comment,0.999912,Others
3,3.0,0.0,b'gAAAAABjLvrbHWyci7Rk4VVhe3pLHyE2adg_o-VOKR7A...,living singapore expensive stressful enough ma...,b'gAAAAABjLvrbH8cm5aYDWCBWr3N_SGc5ZG9SzjZa8w7K...,2021-11-24 18:53:11,comment,0.999640,Compliments
4,4.0,0.0,b'gAAAAABjLvrbbJilS1JbfzeX8yHkEN9GA5jtabKdcoE6...,geraldine ang,b'gAAAAABjLvrbHRlS3tFOSqFUGV1EGNRuh9zP7aseu-ag...,2021-11-24 18:53:12,comment,0.937974,Others
...,...,...,...,...,...,...,...,...,...
699,699.0,0.0,b'gAAAAABjLvrbEV6ANl_r-luHEIY6SSBbpHWwLwJIihG3...,leigh young true none qualified doctor,b'gAAAAABjLvrbFZvJm-XedOtRSTQumLOabL9U9hPYOO9f...,2021-09-08 20:43:30,reply,0.999904,Others
700,700.0,0.0,b'gAAAAABjLvrbnBTI7VWDArf0SqLlSZ1szQVsFbtrGcNe...,john francis head doctor,b'gAAAAABjLvrb3QDLw1OmUoPTNAnCtk5CcZsMDilDYvSp...,2021-09-08 20:43:31,reply,0.999899,Others
701,701.0,0.0,b'gAAAAABjLvrbNCK6isL4MlAkafperQq_lqsC0lou0-Xv...,kelvin chew agree,b'gAAAAABjLvrb1zzEXhvFmvf8dbJb-rT_9lAjWLuwqahA...,2021-09-08 20:43:31,reply,0.999906,Others
702,702.0,0.0,b'gAAAAABjLvrbQq247BRRwaVp7q6jbdddTqyMdK-BXVtH...,kelvin chew,b'gAAAAABjLvrb1Tce3RZShcv6U3UjgeON9dB7jGdce6HY...,2021-09-08 20:43:31,reply,0.999896,Others


In [ ]:
import sentencepiece
from transformers import AdamW, XLNetTokenizer, XLNetModel, XLNetLMHeadModel, XLNetConfig, XLNetForSequenceClassification

# Function to tokenize text according to pre-trained model input,
# then return a sorted list containing the length of the 10 longests texts
def sentence_embeddings_length(text_list, tokenizer):
    tokenized_texts = list(map(lambda t: tokenizer.tokenize(t), text_list))
    tokenized_texts_len = list(map(lambda t: len(t), tokenized_texts))
    return sorted(tokenized_texts_len, reverse=True)[:10]

train_text_list = train["text"].values

# Set the tokenizer to the one used by pre-trained XLNET base case
tokenizer = XLNetTokenizer.from_pretrained('xlnet-base-cased')

# Set the number of tokens for the model training later at the 10th longest text
num_tokens = sentence_embeddings_length(train_text_list, tokenizer)[-1]

Downloading:   0%|          | 0.00/798k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/760 [00:00<?, ?B/s]

In [ ]:
from keras_preprocessing.sequence import pad_sequences

def tokenize_inputs(text_list, tokenizer, num_embeddings=512):
    """
    #Step 1: tokenizes the input text input into ids, which is the input to the model
    input ids: a sequence of integers identifying each input token to its index number in the XLNet tokenizer vocabulary
    
    #Step 2: Appends the special characters to the end of the text to denote end of sentence.
    
    #Step 3: XLNet requires input arrays to be the same size. Select a maximum sentence length.
    Padding and truncating input until every all are of the same length.

    """
    # tokenize the text, then truncate sequence to the desired length minus 2 for the 2 special characters
    tokenized_texts = list(map(lambda t: tokenizer.tokenize(t)[:num_embeddings-2], text_list))

    # convert tokenized text into numeric ids
    input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]

    # append special token "<s>" and </s> to end of sentence
    input_ids = [tokenizer.build_inputs_with_special_tokens(x) for x in input_ids]

    # padding and truncating
    input_ids = pad_sequences(input_ids, maxlen=num_embeddings, dtype="long", truncating="post", padding="post")
    return input_ids

train_input_ids = tokenize_inputs(train_text_list, tokenizer, num_tokens)

In [ ]:
def create_attn_masks(input_ids):
    """
    This is an 'optional' input to the model
    Attention masks tell model whether attention should be applied to the input id tokens
    1 is for actual token, 0 is meant for padding token 
    """
    # Create attention masks
    attention_masks = []

    # Create a mask of 1s for each token followed by 0s for padding
    for seq in input_ids:
        seq_mask = [float(i>0) for i in seq]
        attention_masks.append(seq_mask)
    return attention_masks
    
train_attention_masks = create_attn_masks(train_input_ids)

In [ ]:
# appending the new features to the train df
train["features"] = train_input_ids.tolist()
train["masks"] = train_attention_masks
train.head()

,text,label,features,masks
0,course doctor goh wrong experimental gene ther...,AlternativeCures,"[477, 2223, 216, 409, 1411, 8284, 7104, 4764, ...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ..."
1,also medicine antiparastic drug used india mex...,AlternativeCures,"[77, 3301, 932, 3378, 13996, 1208, 179, 25, 43...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ..."
2,try googling iris koh church iris koh said ha ...,AlternativeCures,"[714, 216, 155, 14870, 17, 8014, 23, 17, 21719...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ..."
3,neo ko hwai ivan know covid case took med plus...,AlternativeCures,"[9731, 17, 1507, 17, 20061, 150, 17, 150, 3207...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ..."
4,take cough syrup day,AlternativeCures,"[182, 12572, 20392, 191, 4, 3, 0, 0, 0, 0, 0, ...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, ..."


In [ ]:
from sklearn import preprocessing

# converting categorial variables into numbers for training
le = preprocessing.LabelEncoder()
le.fit(train['label'])
train['label_categorical'] = le.transform(train['label'])
train.head()

,text,label,features,masks,label_categorical
0,course doctor goh wrong experimental gene ther...,AlternativeCures,"[477, 2223, 216, 409, 1411, 8284, 7104, 4764, ...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",0
1,also medicine antiparastic drug used india mex...,AlternativeCures,"[77, 3301, 932, 3378, 13996, 1208, 179, 25, 43...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",0
2,try googling iris koh church iris koh said ha ...,AlternativeCures,"[714, 216, 155, 14870, 17, 8014, 23, 17, 21719...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",0
3,neo ko hwai ivan know covid case took med plus...,AlternativeCures,"[9731, 17, 1507, 17, 20061, 150, 17, 150, 3207...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",0
4,take cough syrup day,AlternativeCures,"[182, 12572, 20392, 191, 4, 3, 0, 0, 0, 0, 0, ...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, ...",0


In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_validation, X_train_labels, X_validation_labels = train_test_split(train, train['label_categorical'], random_state=1, test_size=0.15, stratify = train['label_categorical'])

# extract masks 
X_train_masks = X_train["masks"].values.tolist() 
X_validation_masks = X_validation["masks"].values.tolist()

# extract labels
X_train_labels = X_train_labels.values.tolist() 
X_validation_labels = X_validation_labels.values.tolist()

# number of labels for train data
num_labels = len(set(X_train_labels))

X_train = X_train["features"].values.tolist() 
X_validation = X_validation["features"].values.tolist()

In [ ]:
# Convert input ids and attention masks into torch tensors which is required input datatype for model
X_train = torch.tensor(X_train)
X_valid = torch.tensor(X_validation)

X_train_labels = torch.tensor(X_train_labels)
X_validation_labels = torch.tensor(X_validation_labels)

X_train_masks = torch.tensor(X_train_masks, dtype=torch.long)
X_validation_masks = torch.tensor(X_validation_masks, dtype=torch.long)

In [ ]:
# Select a batch size for training, reduce if cuda runs out of memory
batch_size = 16

# Create an iterator of our data with torch DataLoader to save memory as the entire dataset does not need to be loaded into memory
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

train_data = TensorDataset(X_train, X_train_masks, X_train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

validation_data = TensorDataset(X_valid, X_validation_masks, X_validation_labels)
validation_sampler = SequentialSampler(validation_data)
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)

In [ ]:
# instantiate a pre-trained XLNET model
model = XLNetForSequenceClassification.from_pretrained("xlnet-base-cased", num_labels=num_labels)
model.cuda()

Downloading:   0%|          | 0.00/467M [00:00<?, ?B/s]

Some weights of the model checkpoint at xlnet-base-cased were not used when initializing XLNetForSequenceClassification: ['lm_loss.weight', 'lm_loss.bias']
- This IS expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLNetForSequenceClassification were not initialized from the model checkpoint at xlnet-base-cased and are newly initialized: ['sequence_summary.summary.bias', 'sequence_summary.summary.weight', 'logits_proj.weight', 'logits_proj.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions a

XLNetForSequenceClassification(
  (transformer): XLNetModel(
    (word_embedding): Embedding(32000, 768)
    (layer): ModuleList(
      (0): XLNetLayer(
        (rel_attn): XLNetRelativeAttention(
          (layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (ff): XLNetFeedForward(
          (layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (layer_1): Linear(in_features=768, out_features=3072, bias=True)
          (layer_2): Linear(in_features=3072, out_features=768, bias=True)
          (dropout): Dropout(p=0.1, inplace=False)
          (activation_function): GELUActivation()
        )
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (1): XLNetLayer(
        (rel_attn): XLNetRelativeAttention(
          (layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (ff): XLNetFeedForward

In [ ]:
# Setting the hyperparameters
param_optimizer = list(model.named_parameters())
no_decay = ['bias', 'gamma', 'beta']
optimizer_grouped_parameters = [
    {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
     'weight_decay_rate': 0.01},
    {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)],
     'weight_decay_rate': 0.0}
]

optimizer = torch.optim.AdamW(optimizer_grouped_parameters, lr=5e-5) # do not exceed 6e-5

In [ ]:
# function to calculate the validation accuracy of 1 batch of predictions
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [ ]:
from sklearn.metrics import f1_score

# function to calculate the validation f1 score of 1 batch of predictions
def flat_f1(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return f1_score(pred_flat, labels_flat, average='weighted')

In [ ]:
# Number of training epochs
epochs = 8

# trange is a tqdm wrapper around the normal python range
# to provide a timing function to show users progress of this code block
from tqdm import tqdm, trange
for _ in trange(epochs, desc="Epoch"):
  
  # Training Mode
  model.train()
  
  # Tracking variables
  tr_loss = 0
  nb_tr_examples, nb_tr_steps = 0, 0
  
  # Train the data for one epoch
  for step, batch in enumerate(train_dataloader):
    # Add batch to GPU
    batch = tuple(t.to(device) for t in batch)
    # Unpack the inputs from our dataloader
    b_input_ids, b_input_mask, b_labels = batch
    # Clear out the gradients (by default they accumulate)
    optimizer.zero_grad()
    # Forward pass
    outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask, labels=b_labels)
    loss = outputs[0]
    logits = outputs[1]
    
    # Backward pass
    loss.backward()
    # Update parameters and perform gradient descent
    optimizer.step()
    
    # Update tracking variables
    tr_loss += loss.item()
    nb_tr_examples += b_input_ids.size(0)
    nb_tr_steps += 1

  print("Train loss: {}".format(tr_loss/nb_tr_steps))
    
    
  # Validation Mode
  model.eval()

  # Tracking variables 
  eval_loss, eval_accuracy, eval_f1 = 0, 0, 0
  nb_eval_steps, nb_eval_examples = 0, 0

  # Evaluate data for one epoch
  for batch in validation_dataloader:
    # Add batch to GPU
    batch = tuple(t.to(device) for t in batch)
    # Unpack the inputs from our dataloader
    b_input_ids, b_input_mask, b_labels = batch
    # Telling the model not to compute or store gradients, optimize for time
    with torch.no_grad():
      # Forward pass, calculate logit predictions
      output = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)
      logits = output[0]
    
    # Move logits and labels to CPU
    logits = logits.detach().cpu().numpy()
    label_ids = b_labels.to('cpu').numpy()

    # Calculation of metrics for one batch
    tmp_eval_accuracy = flat_accuracy(logits, label_ids)
    tmp_eval_f1 = flat_f1(logits, label_ids)
    
    # Aggregation of metrics across all batches
    eval_accuracy += tmp_eval_accuracy
    eval_f1 += tmp_eval_f1
    nb_eval_steps += 1

  print("\nValidation Accuracy: {}".format(eval_accuracy/nb_eval_steps))
  print("Validation f1-score: {}".format(eval_f1/nb_eval_steps))

Epoch:   0%|          | 0/8 [00:00<?, ?it/s]

Train loss: 2.4641483610326596


Epoch:  12%|█▎        | 1/8 [00:31<03:42, 31.80s/it]


Validation Accuracy: 0.15486111111111112
Validation f1-score: 0.2157688492063492
Train loss: 2.2940686659379437


Epoch:  25%|██▌       | 2/8 [01:02<03:07, 31.23s/it]

...


Validation Accuracy: 0.7076388888888889
Validation f1-score: 0.7040688131313132
Train loss: 0.252283190800385


Epoch:  88%|████████▊ | 7/8 [03:44<00:32, 32.16s/it]


Validation Accuracy: 0.6777777777777778
Validation f1-score: 0.6909620009620009
Train loss: 0.21994166475805368


Epoch: 100%|██████████| 8/8 [04:16<00:00, 32.04s/it]


Validation Accuracy: 0.7201388888888889
Validation f1-score: 0.731666366041366


In [ ]:
# process test data using the pre-trained tokenizer
test_text_list = test["text"].values
sentence_embeddings_length(test_text_list, tokenizer)

# process categorical labels, mapping to XLNET ids and creating attention masks for test data
test['label_categorical'] = le.transform(test['label'])
test_input_ids = tokenize_inputs(test_text_list, tokenizer, num_tokens) 
test_attention_masks = create_attn_masks(test_input_ids)

In [ ]:
test["features"] = test_input_ids.tolist()
test["masks"] = test_attention_masks

In [ ]:
# extract masks 
prediction_masks = test["masks"].values.tolist() 
# extract labels
prediction_labels = test["label_categorical"].values.tolist() 
# extract features
prediction = test["features"].values.tolist() 

# conversion of test data into tensor similar to train data
X_test = torch.tensor(prediction)
X_test_labels = torch.tensor(prediction_labels)
X_test_masks = torch.tensor(prediction_masks, dtype=torch.long)

test_data = TensorDataset(X_test, X_test_masks, X_test_labels)
test_sampler = RandomSampler(test_data)
test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=8)

In [ ]:
# Prediction on test set

# Put model in evaluation mode
model.eval()

# Tracking variables 
predictions, true_labels = [], []

# Predict 
for batch in test_dataloader:
  # Add batch to GPU
  batch = tuple(t.to(device) for t in batch)
  # Unpack the inputs from our dataloader
  b_input_ids, b_input_mask, b_labels = batch
  # Telling the model not to compute or store gradients, saving memory and speeding up prediction
  with torch.no_grad():
    # Forward pass, calculate logit predictions
    outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)
    logits = outputs[0]

  # Move logits and labels to CPU
  logits = logits.detach().cpu().numpy()
  label_ids = b_labels.to('cpu').numpy()
  
  # Store predictions and true labels
  predictions.append(logits)
  true_labels.append(label_ids)

In [ ]:
print(predictions[0])
print(true_labels[0])

[[-2.7424512e+00  7.4883932e-01 -1.5336018e+00  3.2710354e+00
   3.1882134e-01  8.2171673e-01  1.3324225e+00  8.0511105e-01
  -1.0384258e+00 -2.0730584e+00 -2.3627596e+00 -8.2217902e-01]
 [-2.5884750e+00  2.4048977e+00 -7.9464573e-01  3.5757213e+00
   2.1303546e+00 -1.9115053e+00 -3.1342339e-02  1.7495679e-02
  -7.9394710e-01  1.6674110e+00 -1.7746352e+00 -2.2048464e+00]
 [-1.1911381e+00 -1.2180171e+00 -1.8345796e+00 -2.5709996e+00
  -2.1232706e-01 -1.2780396e+00  1.4433752e+00  2.5779033e+00
  -3.2492700e-01 -1.4390335e+00 -2.1327255e+00  2.8757002e+00]
 [-1.0308484e+00  1.6370194e+00 -4.6871039e-01  1.0705067e+00
   2.6468736e-01  5.9641523e+00  6.5377498e-01 -1.7811756e+00
  -1.9602951e+00 -3.0785439e+00 -6.7442054e-01  6.8191308e-01]
 [-8.9565702e-02  4.1978054e-02 -1.3707136e+00  3.4958467e+00
   6.6627769e+00  4.1133631e-02 -1.2289157e+00 -1.0476042e+00
  -8.8765037e-01 -1.5562261e+00 -2.1283286e+00 -6.2057092e-03]
 [-1.5703862e+00 -4.3149465e-01 -9.3858600e-01  6.9305506e+00
   

In [ ]:
from scipy.special import softmax

# this converts the result across batches to a flat array  
pred_labels = []
pred_prob = []
for i in range(len(true_labels)):
  probabilities = softmax(predictions[i], axis=1)
  y = np.argmax(predictions[i], axis=1)
  for j in range(len(y)):
    pred_labels.append(y[j])
    pred_prob.append(probabilities[j][y[j]])

In [ ]:
# converting the numerical labels back into categorical variables for human understanding
xlnet_labels = le.inverse_transform(pred_labels)

In [ ]:
diagdf = pd.read_excel(test_set_path)

# add 2 columns, xlnet label and probability to the BERT labelled test data for comparison
diagdf["xlnet_label"] = xlnet_labels
diagdf["xlnet_prob"] = pred_prob

# add a column to find difference between probability of XLNET - BERT
diagdf["modeldiff"] = diagdf["xlnet_prob"] - diagdf["comment_pred_prob"]

# add a column for matching labels
diagdf["match"] = np.where((diagdf['label'] == diagdf['xlnet_label']), 0, 1) #matching is 0

# sort first by matching, matched rows goes to the bottom
# then sort by probability difference, descending order
diagdf = diagdf.sort_values(by=['match','modeldiff'], ascending=False, ignore_index=True)
diagdf.head(10)

,Unnamed: 0,group,post_id,text,comment_id,comment_time,type,comment_pred_prob,label,xlnet_label,xlnet_prob,modeldiff,match
0,431.0,0.0,b'gAAAAABjLvrbwR7NU-302O3seH_UResZeUHrKslo0rfu...,True facts in life. Who r u following?,b'gAAAAABjLvrbKDMBUPgiDBBsUigD7yt18Szn5EMLS7lS...,2021-09-08 20:42:53,comment,0.318054,Vaccines_TradSuperior,Conspiracy_Control&Manip,0.994097,0.676043,1
1,109.0,0.0,b'gAAAAABjLvrbSZK86sjOOFtFkr_qIAbDI-z0owW1Pcqr...,THE JAB MUST BE STOPPED IMMEDIATELY!!! https:/...,b'gAAAAABjLvrboxF4v4505V5Q6Y51_QCcB-kquBJ0HcIq...,2021-11-24 18:54:30,comment,0.410454,Others,AlternativeCures,0.998319,0.587866,1
2,38.0,0.0,b'gAAAAABjLvrbUvX18RWXTTaX4Y2D-Tu0Jz_BldvNCsWh...,Derek Loui,b'gAAAAABjLvrbAx-Ip4eg21L0lRR5a2snjHQcqJR2ozQp...,2021-11-17 18:53:22,comment,0.386237,Others,Compliments,0.962154,0.575918,1
3,515.0,0.0,b'gAAAAABjLvrbqssHrUS3ugT_S8ucMtNL8CTWQCNBmFVf...,Derek Loui,b'gAAAAABjLvrbzO3W0nguEuwZUUOllJeSfj2Ycu39H3Ar...,2021-12-04 00:00:00,reply,0.386237,Others,Vaccines_HealthRisk,0.945733,0.559496,1
4,162.0,0.0,b'gAAAAABjLvrbKY5FDqP65WGflLnkbx60fyrg9iUNLZUD...,THE JAB MUST BE STOPPED IMMEDIATELY!!! https:...,b'gAAAAABjLvrbaFaL5MA9K3_NneluoaTZvROwHRyal8iR...,2021-11-24 18:55:30,comment,0.410454,Others,Compliments,0.963969,0.553515,1
5,408.0,0.0,b'gAAAAABjLvrbiAe4MiwTrjxueSESBOSMuflwyrPjGqjb...,https:// youtu.be/ pENijjYcwBw,b'gAAAAABjLvrb-r6HGXjPYyJmygUe9CZ3DDEla_KCqGG7...,2021-09-08 20:42:38,comment,0.395520,Conspiracy_VirusOrigins,Others,0.943643,0.548123,1
6,362.0,0.0,b'gAAAAABjLvrb2J7EKZ7XfWkasK2ujJeRTlS-mdPWOW4x...,https:// youtu.be/ IQAeguvHFM8,b'gAAAAABjLvrbsgoWXzBcPYG5h1yCaJ-eeyBKWphdhEKQ...,2021-09-08 19:32:42,comment,0.433185,Vaccines_HealthRisk,VirusMisinformation,0.980400,0.547215,1
7,614.0,0.0,b'gAAAAABjLvrb9-sZMjAmr0zoc19NLGxHMPD9611rFBTS...,Pam Cheong Cheong They're wrong. Trump's failu...,b'gAAAAABjLvrb1rnSosn47P-7eImWC0wtnpJR2eQe9ndM...,2021-11-08 18:59:07,reply,0.415077,Complaints,Compliments,0.952069,0.536992,1
8,145.0,0.0,b'gAAAAABjLvrbVfjlMUwKc_IX172OT_06HXYWM_FGdIpY...,Their Logic is Conform to their Vaccine progra...,b'gAAAAABjLvrbS6oT4UDNxC5Ede3NWQWxWtK32IdmwPB-...,2021-11-24 18:54:51,comment,0.412673,Conspiracy_ConcealingTruth,Vaccines_HealthRisk,0.937980,0.525307,1
9,263.0,0.0,b'gAAAAABjLvrbhGWi99QNy6sJA565T4R82K_OsAG9xHbO...,Why do you want to know?,b'gAAAAABjLvrb3NZvWds32FGTheXfh7kw14-iz3pbzJaf...,2021-11-08 18:58:59,comment,0.482116,VirusMisinformation,Vaccines_Ineffective,0.986189,0.504073,1


In [ ]:
# check percentile of BERT prediction probabilities
diagdf["comment_pred_prob"].quantile([.05, .1, .15, .2], interpolation="nearest")

0.05    0.546498
0.10    0.672492
0.15    0.837520
0.20    0.935413
Name: comment_pred_prob, dtype: float64

In [ ]:
tenthpercentile = diagdf["comment_pred_prob"].quantile(.1, interpolation="nearest")
print(tenthpercentile)

0.672491788864135


In [ ]:
# moving the worst 10 percent performing on BERT to another copy, to be added back later
lowprob = list(np.where(diagdf['comment_pred_prob'] <= tenthpercentile)[0]) 
lowprobmatch = set(np.where(diagdf['match'] < 1)[0])

# make sure agreements not inside low prob of BERT (duplicate issue prevented)
idxbreak = 0
for idx, x in enumerate(lowprob):
  if x in lowprobmatch:
    idxbreak = idx
    break

lowprob = lowprob[:idxbreak]
lowprobdf = diagdf[diagdf.index.isin(lowprob)] # make a copy of low prob
maindf = diagdf.drop(lowprob)

# reordering the 2 dfs
finaldf = pd.concat([lowprobdf, maindf], ignore_index=True)
finaldf = finaldf.drop(['modeldiff', 'match'], axis = 1)
finaldf

,Unnamed: 0,group,post_id,text,comment_id,comment_time,type,comment_pred_prob,label,xlnet_label,xlnet_prob
0,431.0,0.0,b'gAAAAABjLvrbwR7NU-302O3seH_UResZeUHrKslo0rfu...,True facts in life. Who r u following?,b'gAAAAABjLvrbKDMBUPgiDBBsUigD7yt18Szn5EMLS7lS...,2021-09-08 20:42:53,comment,0.318054,Vaccines_TradSuperior,Conspiracy_Control&Manip,0.994097
1,109.0,0.0,b'gAAAAABjLvrbSZK86sjOOFtFkr_qIAbDI-z0owW1Pcqr...,THE JAB MUST BE STOPPED IMMEDIATELY!!! https:/...,b'gAAAAABjLvrboxF4v4505V5Q6Y51_QCcB-kquBJ0HcIq...,2021-11-24 18:54:30,comment,0.410454,Others,AlternativeCures,0.998319
2,38.0,0.0,b'gAAAAABjLvrbUvX18RWXTTaX4Y2D-Tu0Jz_BldvNCsWh...,Derek Loui,b'gAAAAABjLvrbAx-Ip4eg21L0lRR5a2snjHQcqJR2ozQp...,2021-11-17 18:53:22,comment,0.386237,Others,Compliments,0.962154
3,515.0,0.0,b'gAAAAABjLvrbqssHrUS3ugT_S8ucMtNL8CTWQCNBmFVf...,Derek Loui,b'gAAAAABjLvrbzO3W0nguEuwZUUOllJeSfj2Ycu39H3Ar...,2021-12-04 00:00:00,reply,0.386237,Others,Vaccines_HealthRisk,0.945733
4,162.0,0.0,b'gAAAAABjLvrbKY5FDqP65WGflLnkbx60fyrg9iUNLZUD...,THE JAB MUST BE STOPPED IMMEDIATELY!!! https:...,b'gAAAAABjLvrbaFaL5MA9K3_NneluoaTZvROwHRyal8iR...,2021-11-24 18:55:30,comment,0.410454,Others,Compliments,0.963969
...,...,...,...,...,...,...,...,...,...,...,...
699,595.0,0.0,b'gAAAAABjLvrbkUfgmkYw1N8a1nWWgoOpCxqz4Ccng-sO...,Wayne Tan Exactly ...,b'gAAAAABjLvrb231-DO0Ws1aJABE7WtXCPzevJixpUPat...,2021-11-08 18:58:54,reply,0.999876,Others,Others,0.472939
700,155.0,0.0,b'gAAAAABjLvrb53nqAEREcgc3uw84zzUu7PoawEDqy3p-...,"Dr. Ruben on the FDA panel: ""We're never gonna...",b'gAAAAABjLvrbboo4eh04RJFXvBiDkEQS3NyY47YFd7Ci...,2021-11-24 18:55:29,comment,0.999667,Vaccines_Ineffective,Vaccines_Ineffective,0.448810
701,105.0,0.0,b'gAAAAABjLvrbov0A67DplpozymByfbzH3chEJyyWdvOh...,"The opening words ""voluntarily unvaccinated"" i...",b'gAAAAABjLvrb5EXUtGLwBxleX8ysluBXJn--rgMhUGxV...,2021-11-24 18:54:28,comment,0.999838,Complaints,Complaints,0.391027
702,376.0,0.0,b'gAAAAABjLvrbhy4FFeyTNUkZYKSinaEttRCNf4z7OZ_k...,Remember even if vaccinated there are no signs...,b'gAAAAABjLvrb_EAjf-hL0B-46ZOiHug_AAOktGOQnocN...,2021-09-08 19:32:59,comment,0.997981,Complaints,Complaints,0.367947


In [ ]:
# convert the final presentation order to an excel to be downloaded
finaldf.to_excel("output.xlsx")  